In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
synthetic_df = pd.read_csv('/content/drive/Shareddrives/CMPUT 664 : SSE/Data/synthetic_data_adult_census.csv')


In [ ]:
categorical_columns = ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race','sex', 'native_country']
for column in categorical_columns:
    tempdf = pd.get_dummies(synthetic_df[column], prefix=column)
    synthetic_df = pd.merge(
        left=synthetic_df,
        right=tempdf,
        left_index=True,
        right_index=True,
    )
    synthetic_df = synthetic_df.drop(columns=column)

In [ ]:
x = synthetic_df.iloc[:, :-1].to_numpy()
y = synthetic_df['<50k'].to_numpy()

In [ ]:
from typing import List, Tuple, Dict
from copy import copy

from tqdm import tqdm_notebook

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.base import clone, BaseEstimator

try:
    import tensorflow as tf
except (ModuleNotFoundError, ImportError):
    import warnings

    warnings.warn("Tensorflow is not installed")

class ShadowModels:
    """
    Creates a swarm of shadow models and trains them with a split
    of the synthetic data.
    Parameters
    ----------
    X: ndarray or DataFrame
    y: ndarray or str
        if X it's a DataFrame then y must be the target column name,
        otherwise 
    n_models: int
        number of shadow models to build. Higher number returns
        better results but is limited by the number of records 
        in the input data.
    target_classes: int
        number of classes of the target model or lenght of the
        prediction array of the target model.
    learner: learner? #fix type
        learner to use as shadow model. It must be as similar as 
        possible to the target model. It must have `predict_proba` 
        method. Now only sklearn learners are implemented.
    Returns
    -------
    ShadowModels object
    """

    def __init__(
        self,
        X: np.ndarray,
        y: np.ndarray,
        n_models: int,
        target_classes: int,
        learner,
        **fit_kwargs,
    ) -> None:

        self.n_models = n_models
        self.X = X
        if self.X.ndim > 1:
            # flatten images or matrices inside 1rst axis
            self.X = self.X.reshape(self.X.shape[0], -1)

        self.y = y
        self.target_classes = target_classes
        self._splits = self._split_data(self.X, self.y, self.n_models, self.target_classes)
        self.learner = learner
        self.models = self._make_model_list(self.learner, self.n_models)

        # train models
        self.results = self.train_predict_shadows(**fit_kwargs)

    @staticmethod
    def _split_data(
        X: np.ndarray, y: np.ndarray, n_splits: int, n_classes: int
    ) -> List[np.ndarray]:
        """
        Split manually into n datasets maintaining class proportions
        """
        # data = np.hstack((data[0], data[1].reshape(-1, 1)))
        # X = data
        # y = data[:, -1]
        classes = range(n_classes)
        class_partitions = []
        # Split by class
        for clss in classes:

            X_clss = X[y == clss]
            y_clss = y[y == clss]
            batch_size = len(X_clss) // n_splits
            splits = []
            for i in range(n_splits):
                split_X = X_clss[i * batch_size : (i + 1) * batch_size, :]
                split_y = y_clss[i * batch_size : (i + 1) * batch_size]
                splits.append(np.hstack((split_X, split_y.reshape(-1, 1))))
            class_partitions.append(splits)

        # -------------------
        # consolidate splits into ndarrays
        # -------------------

        grouped = []
        for split in range(n_splits):
            parts = []
            for part in class_partitions:
                parts.append(part[split])
            grouped.append(parts)

        splits = []
        for group in grouped:
            splits.append(np.vstack(group))

        return splits

    @staticmethod
    def _make_model_list(learner, n) -> List:
        """
        Intances n shadow models, copies of the input parameter learner
        """
        try:
            if isinstance(learner, tf.keras.models.Model):
                models = [copy(learner) for _ in range(n)]
        except NameError:
            print("using sklearn shadow models")
            pass

        if isinstance(learner, BaseEstimator):
            models = [clone(learner) for _ in range(n)]

        return models

    def train_predict_shadows(self, **fit_kwargs):
        """
        "in" : 1
        "out" : 0
        """

        # TRAIN and predict
        results = []
        for model, data_subset in tqdm_notebook(zip(self.models, self._splits)):
            X = data_subset[:, :-1]
            y = data_subset[:, -1]
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

            model.fit(X_train, y_train, **fit_kwargs)
            # data IN training set labelet 1
            y_train = y_train.reshape(-1, 1)
            predict_in = model.predict_proba(X_train)
            res_in = np.hstack((predict_in, y_train, np.ones_like(y_train)))

            # data OUT of training set, labeled 0
            y_test = y_test.reshape(-1, 1)
            predict_out = model.predict_proba(X_test)
            print(predict_out)
            res_out = np.hstack((predict_out, y_test, np.zeros_like(y_test)))

            # concat in single array
            model_results = np.vstack((res_in, res_out))
            results.append(model_results)

        results = np.vstack(results)
        return results

    def __repr__(self):
        rep = (
            f"Shadow models: {self.n_models}, {self.learner.__class__}\n"
            f"lengths of data splits : {[len(s) for s in self._splits]}"
        )
        return rep

In [ ]:
logisticRegr = LogisticRegression()
sh = ShadowModels(x, y, 5, 3, logisticRegr)
shadow_data = sh.results

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:134: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

[[0.80695171 0.19304829]
 [0.72768247 0.27231753]
 [0.82224103 0.17775897]
 ...
 [0.88957846 0.11042154]
 [0.72533553 0.27466447]
 [0.78438407 0.21561593]]
[[0.74082108 0.25917892]
 [0.92842259 0.07157741]
 [0.89419311 0.10580689]
 ...
 [0.83816263 0.16183737]
 [0.66061862 0.33938138]
 [0.84336818 0.15663182]]
[[0.30536151 0.69463849]
 [0.51021045 0.48978955]
 [0.88698643 0.11301357]
 ...
 [0.83181384 0.16818616]
 [0.62801964 0.37198036]
 [0.81117156 0.18882844]]
[[0.87026372 0.12973628]
 [0.65759953 0.34240047]
 [0.89258246 0.10741754]
 ...
 [0.82106035 0.17893965]
 [0.95720443 0.04279557]
 [0.81443442 0.18556558]]
[[0.90529939 0.09470061]
 [0.64929656 0.35070344]
 [0.30154166 0.69845834]
 ...
 [0.61764529 0.38235471]
 [0.34926376 0.65073624]
 [0.86677005 0.13322995]]


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [ ]:
shadow_data

array([[0.77467677, 0.22532323, 1.        , 1.        ],
       [0.80815934, 0.19184066, 0.        , 1.        ],
       [0.78030288, 0.21969712, 0.        , 1.        ],
       ...,
       [0.61764529, 0.38235471, 0.        , 0.        ],
       [0.34926376, 0.65073624, 0.        , 0.        ],
       [0.86677005, 0.13322995, 0.        , 0.        ]])

In [ ]:
class AttackModels:
    def __init__(self, target_classes, attack_learner):
        """
        Attacker models to learn class membership from shadow data.
        Parameters
        ----------
        target_classes: int
            number of classes that the target model can predict
        attack_learning: learner
            trainable learner to model memebership from shadow data.
            The learner its cloned into n models, one for each target class,
            and each model is trained on a class subset of the shadow data.
        Returns
        -------
        AttackModels class instance
        """
        self.target_classes = target_classes
        self.attack_learner = attack_learner
        # 1 model for each class
        self.attack_models = [clone(self.attack_learner) for _ in range(target_classes)]

        self._fited = False

    @staticmethod
    def _update_learner_params(learner, **learner_params) -> None:
        # safety check if dict is well formed
        for k in learner_params.keys():
            if not hasattr(learner, k):
                raise AttributeError(
                    f"Learner parameter {k} is not an attribute of {learner.__class__}"
                )

        # update learner params
        learner.__dict__.update(**learner_params)

    def fit(self, shadow_data, **learner_kwargs) -> None:
        """
        Trains `attack_models` with `shadow_data`. Each model is trained with
        with a subset of the same class of `shadow_data`.
        Parameters
        ----------
        shadow_data: np.ndarray
            Shadow data. Results from `ShadowModels`.
            Last column (`[:,-1]`) must be the membership label of the shadow
            prediction, where 1 means that the record was present in the 
            shadow training set ('in') and 0 if the recored was in the test
            set ('out').
            Second last column (`[:,-2]`) must be the data class. this will
            be used as grouper to split the data for each attack model.
            The rest of the columns are the class probability vector
            predicted by the shadow model.
        Returns
        -------
        None
        TODO
        ----
            Tweak model params with something like **learner_kwargs
            cross-validate
            grid search?
        """
        # split data into subsets, n == target_classes
        membership_label = shadow_data[:, -1]
        class_label = shadow_data[:, -2]
        data = shadow_data[:, :-2]
        for i, model in enumerate(self.attack_models):
            X = data[class_label == i]
            y = membership_label[class_label == i]

            # update model params
            self._update_learner_params(model, **learner_kwargs)
            # train model
            model.fit(X, y)

        self._fited = True

    def predict(self, X, y, batch=False) -> np.ndarray:
        """
        Predicts if `X` is real member of `y` in the attacked
        private training set.
        Parameters
        ----------
        X: np.ndarray
            Probability vector result from target model
        y: int, np.ndarray
            estimated class of the data record used to get `X`
        """
        if not self._fited:
            print("Must run `fit` method first")
            return

        if not batch:
            model_cls = y
            model = self.attack_models[model_cls]
            prob_vec = model.predict_proba(X)

            if y == np.argmax(prob_vec) and np.argmax(prob_vec) == 1:
                return 1

            else:
                return 0

        elif batch:

            model_classes = np.unique(y)
            res = []
            for model_cls in model_classes:
                X_cls = X[y == model_cls]
                model = self.attack_models[model_cls]
                attack_res = model.predict_proba(X_cls)
                res.append(attack_res)

            return np.concatenate(res)

In [ ]:
model = RandomForestClassifier(n_estimators=100)
attacker = AttackModels(target_classes=2, attack_learner=model)
attacker.fit(shadow_data)

# **Membership attack on target Decision Tree**

In [ ]:
from sklearn.svm import SVC
df = pd.read_csv('/content/drive/Shareddrives/CMPUT 664 : SSE/Data/adult census/adult.data',header= None,delimiter=",")
df[14] = np.where(df[14]==' <=50K', 0, 1)
x = df.drop(14,axis = 1 )
y = df[14]
x = pd.get_dummies(x)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.34, random_state=42)
clf = DecisionTreeClassifier(random_state=0)
#clf = make_pipeline(StandardScaler(), SVC(kernel='poly', degree=3, probability = True))
clf.fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


DecisionTreeClassifier(random_state=0)

In [ ]:
y_pred_train = clf.predict(x_train)
train_a = accuracy_score(y_train, y_pred_train, normalize=True)*100
print('accuracy on train data is',train_a)

y_pred_test = clf.predict(x_test)
test_a = accuracy_score(y_test, y_pred_test, normalize=True)*100
print('accuracy on test data is',test_a)


accuracy on train data is 100.0
accuracy on test data is 81.7180019871737


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [ ]:
X_in = clf.predict_proba(x_train)
res_in = attacker.predict(X_in, y_train, batch=True)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [ ]:
X_in

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [ ]:
res_in

array([[0.88      , 0.12      ],
       [0.88      , 0.12      ],
       [0.88      , 0.12      ],
       ...,
       [0.48590858, 0.51409142],
       [0.48590858, 0.51409142],
       [0.48590858, 0.51409142]])

In [ ]:
X_out = clf.predict_proba(x_test)
res_out = attacker.predict(X_out, y_test, batch=True)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [ ]:
X_out

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
y_pred = np.concatenate((np.argmax(res_in, axis=1), np.argmax(res_out, axis=1)))
y_true = np.concatenate((np.ones_like(y_train), np.zeros_like(y_test)))

In [ ]:
precision_score(y_true, y_pred)

0.7566076096427534

In [ ]:
recall_score(y_true, y_pred)

0.2424383434155421

In [ ]:
f1_score(y_true, y_pred)

0.3672117282210318

# **Membership attack on target Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
r_clf = RandomForestClassifier(max_depth=10, n_estimators = 2, random_state=0)
r_clf.fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


RandomForestClassifier(max_depth=10, n_estimators=2, random_state=0)

In [ ]:
y_pred_train = r_clf.predict(x_train)
train_a = accuracy_score(y_train, y_pred_train, normalize=True)*100
print('accuracy on train data is',train_a)

y_pred_test = r_clf.predict(x_test)
test_a = accuracy_score(y_test, y_pred_test, normalize=True)*100
print('accuracy on test data is',test_a)

accuracy on train data is 85.82131223825034
accuracy on test data is 84.52714298618011


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [ ]:
X_in = r_clf.predict_proba(x_train)
res_input = attacker.predict(X_in, y_train, batch=True)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [ ]:
X_in

array([[0.83971705, 0.16028295],
       [0.98548538, 0.01451462],
       [0.98548538, 0.01451462],
       ...,
       [0.98548538, 0.01451462],
       [0.83971705, 0.16028295],
       [0.84149272, 0.15850728]])

In [ ]:
res_input

array([[0.16, 0.84],
       [0.07, 0.93],
       [0.07, 0.93],
       ...,
       [0.87, 0.13],
       [0.86, 0.14],
       [0.87, 0.13]])

In [ ]:
X_out = r_clf.predict_proba(x_test)
res_output = attacker.predict(X_out, y_test, batch=True)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [ ]:
res_output

array([[0.39, 0.61],
       [0.95, 0.05],
       [0.11, 0.89],
       ...,
       [0.99, 0.01],
       [0.  , 1.  ],
       [0.93, 0.07]])

In [ ]:
X_out

array([[0.98029984, 0.01970016],
       [0.54468698, 0.45531302],
       [0.28070029, 0.71929971],
       ...,
       [0.99025628, 0.00974372],
       [0.99057818, 0.00942182],
       [0.96483051, 0.03516949]])

In [ ]:
y_pred = np.concatenate((np.argmax(res_input, axis=1), np.argmax(res_output, axis=1)))
y_true = np.concatenate((np.ones_like(y_train), np.zeros_like(y_test)))

In [ ]:
precision_score(y_true, y_pred)

0.6622998544395924

In [ ]:
recall_score(y_true, y_pred)

0.5504885993485342

In [ ]:
f1_score(y_true, y_pred)

0.601240089449075